In [50]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-133860
Azure region: southcentralus
Subscription id: 9a7511b8-150f-4a58-8528-3e7d50216c31
Resource group: aml-quickstarts-133860


In [51]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

amlcompute_cluster_name= "cpu-cluster"
compute_config = AmlCompute.provisioning_configuration(vm_size = "Standard_D2_V2", max_nodes= 4 )
aml_compute= ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)
aml_compute.wait_for_completion(show_output=True)

Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [56]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform
import os


# Specify parameter sampler
ps = ps = RandomParameterSampling(parameter_space={'C': uniform(0.1, 100)} )
    

# Specify a Policy
policy = BanditPolicy(evaluation_interval=100, slack_factor= 0.2)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
est = SKLearn(source_directory= os.path.join('./training'),entry_script='train.py', compute_target= aml_compute)                                                     

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(estimator= est,
                                    hyperparameter_sampling= ps, 
                                    policy= policy,
                                    primary_metric_name= 'Accuracy', 
                                    primary_metric_goal= PrimaryMetricGoal.MAXIMIZE, 
                                    max_total_runs= 20,
                                    max_concurrent_runs=4)

In [57]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

hdr= exp.submit(hyperdrive_config)
RunDetails(hdr).show()
hdr.wait_for_completion(show_output= True)
assert(hdr.get_status() =="completed")

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_ef520b03-ecc0-41bf-b53e-af2e210f2541
Web View: https://ml.azure.com/experiments/udacity-project/runs/HD_ef520b03-ecc0-41bf-b53e-af2e210f2541?wsid=/subscriptions/9a7511b8-150f-4a58-8528-3e7d50216c31/resourcegroups/aml-quickstarts-133860/workspaces/quick-starts-ws-133860

Streaming azureml-logs/hyperdrive.txt

"<START>[2021-01-07T20:06:48.324921][API][INFO]Experiment created<END>\n""<START>[2021-01-07T20:06:49.106410][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space<END>\n""<START>[2021-01-07T20:06:49.464010][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.<END>\n"<START>[2021-01-07T20:06:50.6986566Z][SCHEDULER][INFO]The execution environment is being prepared. Please be patient as it can take a few minutes.<END>

Execution Summary
RunId: HD_ef520b03-ecc0-41bf-b53e-af2e210f2541
Web View: https://ml.azure.com/experiments/udacity-project/runs/HD_ef520b03-ecc0-41bf-b53e-af2e210f2541?wsid=/subscriptions/9a751

AssertionError: 

In [ ]:
import joblib
# Get your best run and save the model from that run.

best_run= hdr.get_best_run_by_primary_metric()
print(best_run.get_details()['runDefinition']['arguements'])
print(best_run.get_file_names())
model= best_run.register_model(model_name= 'hd_best_model', model_path= 'outputs/model.joblib')

In [31]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

ds= TabularDatasetFactory.from_delimited_files('https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv' )

In [33]:
import pandas as pd

def clean_data(data):
    # Dict for cleaning data
    months = {"jan":1, "feb":2, "mar":3, "apr":4, "may":5, "jun":6, "jul":7, "aug":8, "sep":9, "oct":10, "nov":11, "dec":12}
    weekdays = {"mon":1, "tue":2, "wed":3, "thu":4, "fri":5, "sat":6, "sun":7}

    # Clean and one hot encode data
    x_df = data.to_pandas_dataframe().dropna()
    jobs = pd.get_dummies(x_df.job, prefix="job")
    x_df.drop("job", inplace=True, axis=1)
    x_df = x_df.join(jobs)
    x_df["marital"] = x_df.marital.apply(lambda s: 1 if s == "married" else 0)
    x_df["default"] = x_df.default.apply(lambda s: 1 if s == "yes" else 0)
    x_df["housing"] = x_df.housing.apply(lambda s: 1 if s == "yes" else 0)
    x_df["loan"] = x_df.loan.apply(lambda s: 1 if s == "yes" else 0)
    contact = pd.get_dummies(x_df.contact, prefix="contact")
    x_df.drop("contact", inplace=True, axis=1)
    x_df = x_df.join(contact)
    education = pd.get_dummies(x_df.education, prefix="education")
    x_df.drop("education", inplace=True, axis=1)
    x_df = x_df.join(education)
    x_df["month"] = x_df.month.map(months)
    x_df["day_of_week"] = x_df.day_of_week.map(weekdays)
    x_df["poutcome"] = x_df.poutcome.apply(lambda s: 1 if s == "success" else 0)

    y_df = x_df.pop("y").apply(lambda s: 1 if s == "yes" else 0)
    
    return x_df, y_df

In [34]:
#from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

In [36]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    compute_target= aml_compute
    experiment_timeout_minutes=30,
    task= "classification",
    primary_metric= 'AUC_weighted',
    training_data=x,
    label_column_name=y,
    n_cross_validations=5)

In [37]:
# Submit your automl run

remote_run= exp.submit(automl_config,show_output= False)

ConfigException: ConfigException:
	Message: Argument [label_column_name] is of unsupported type: [<class 'pandas.core.series.Series'>]. Supported type(s): [int, str]
	InnerException: None
	ErrorResponse 
{
    "error": {
        "code": "UserError",
        "message": "Argument [label_column_name] is of unsupported type: [<class 'pandas.core.series.Series'>]. Supported type(s): [int, str]",
        "details_uri": "https://aka.ms/AutoMLConfig",
        "target": "label_column_name",
        "inner_error": {
            "code": "BadArgument",
            "inner_error": {
                "code": "ArgumentInvalid"
            }
        },
        "reference_code": "061ed905-e59e-42b9-ad95-cd18f40b4358"
    }
}

In [ ]:
# Retrieve and save your best automl model.

import joblib
# Get your best run and save the model from that run.

best_run= remote_run.get_best_run_by_primary_metric()
print(best_run.get_details()['runDefinition']['arguements'])

print(best_run.get_file_names())
model= best_run.register_model(model_name= 'AML_best_model', model_path= 'outputs/model.joblib')